# DGL中的消息传递（message passing） 

## 什么是message passing

《Neural Message Passing for Quantum Chemistry》一文首先提出了message passing的框架，其基本思想是：对于图中任一节点$v$, 可以通过聚合$v$的邻居的信息计算$v$的隐含层状态。
下图展示了在有向图和无向图中message passing的过程。对于有向图，信息从源节点传递到汇节点；
对于无向图，由于边没有方向，因此两节点之间只要有边就会进行信息传递。我们也可以把每条无向边看作两条有向边，按照有向图的流程理解message passing在无向图中传播的方向。

<img src='mp.png' width=100%>

## DGL的消息传递范式
对于节点$v$, 假设其特征$x_v \in \mathbb{R^{d_1}}$, 边$(u, v)$上的特征为$w_e \in  \mathbb{R^{d_2}}$。消息传递范式定义了以下节点和边上的计算：

边上计算：${m_e}^{(t+1)} = \phi({x_v}^{(t)}, {x_u}^{(t)}, {m_e}^{(t)}), (u, v, e) \in \varepsilon $.

点上计算：${x_v}^{(t+1)} = \psi({x_v}^{(t)}, \rho( \{{m_e}^{(t+1)},(u, v, e) \in \varepsilon \} ))$.


在等式中，$\phi$是定义在边上的消息函数，它通过将边上的特征${m_e}^{(t)}$与其两端的节点特征${x_v}^{(t)}, {x_u}^{(t)}$结合起来产生消息，最简单的$\phi$可以选择sum，此时的边上计算公式变为${m_e}^{(t+1)} = {x_v}^{(t)} + {x_u}^{(t)} + {m_e}^{(t)},(u, v, e) \in \varepsilon $。对于节点上的计算，$\rho$是聚合函数，它聚合节点$v$的邻居信息，$\psi$是更新函数，它会结合聚合后的信息和节点本省的特征来更新节点特征。

##  DGL消息传递计算的API

### apply_edges

因为在图上，节点的邻居数量是不确定的。所以对于MP的模式，我们不太容易使用张量计算的方式来做。如果对节点用for循环，速度会非常得慢。对此，DGL设计了专门的API来方便进行消息传递范式的计算。

其中针对图中边的计算`apply_edges`是最基础的API之一，关于这个API的文档说明在[这里](https://docs.dgl.ai/generated/dgl.DGLGraph.apply_edges.html?highlight=apply_edges)。

`apply_edges`使用特定的计算函数，比如DGL内置的一些函数（`dgl.function`），或者用户自定义的函数，来完成针对边上特征的计算。DGL提供了很多内置的计算函数，比如`u_add_v`,`u_dot_e`等等，具体可参考[nn.function](https://docs.dgl.ai/api/python/dgl.function.html#dgl-built-in-function)。

下面这个例子里，我们实现了把源节点和目标节点上的特征（key是'h',value是2维，都是1）相加，并放到边上作为特征(key是'm')。

In [6]:
import dgl
import torch as th

g = dgl.graph(([0,1,2,0,1], [3,2,1,2,0]))
g.ndata['h'] = th.ones(4,2)
g.apply_edges(dgl.function.u_add_v('h', 'h', 'm'))
print(g.edata['m'])

tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])


### update_all

因为在图上，节点的邻居数量是不确定的。所以对于MP的模式，我们不太容易使用张量计算的方式来做。如果对节点用for循环，速度会非常得慢。对此，DGL设计了专门的API来方便进行消息传递范式的计算。

其中最常用的API就是`dgl.DGLGraph.update_all()`，关于这个API的文档说明在[这里](https://docs.dgl.ai/generated/dgl.DGLGraph.update_all.html?highlight=update_all#)。

`update_all`接口里有两个必须的函数：`message_func`和`reduce_func`， 其调用方式为`graph.update_all(message_func, reduce_func)`。

`message_func`（消息函数）实现的是边上计算的步骤，它接受一个参数 `edges`，这是一个`EdgeBatch`的实例，在消息传递时，它被dgl在内部生成以表示一批边。 `edges`有`src`、 `dst`和`data`共3个成员属性，分别用于访问源节点、目标节点和边的特征。

用户可以根据需要实现的边上计算公式，设计不同的`message_func`，例如，若我们在边上计算步骤希望实现把源节点（src node)的特征`hu`与目标节点（dst node）的特征`hv`相加，可以定义如下的`message_func`，其输入为edges（dgl的`EdgeBatch`实例），输出为一个字典，键为`he`（我们自定义的名称），值为计算出的`edges`的feature。在dgl中，消息的内置函数的命名约定是`u`表示源节点， `v`表示目标节点，`e`表示边。

In [1]:
def message_func(edges):
     return {'he': edges.src['hu'] + edges.dst['hv']}

该函数等价于 `dgl.function.u_add_v('hu', 'hv', 'he')`，第一二个参数分别为src和dst node的特征名称，第三个参数表示计算得到的feture被储存在`he`这个键对应的值中。

`reduce_func`（聚合函数）实现的是点上计算的步骤，它接受一个参数`nodes`，这是一个`NodeBatch`的实例， 在消息传递时，它被DGL在内部生成以表示一批节点。
`nodes`的成员属性`mailbox`可以用来访问节点收到的消息。 一些最常见的聚合操作包括 sum、max、min 等。
同理，用户可以自定义该函数，如下定义的`reduce_func`实现了将节点收到的信息相加，得到节点新的状态，并存储到`h`这个feature中。

In [1]:
import torch as th
def reduce_func(nodes):
     return {'h': th.sum(nodes.mailbox['m'], dim=1)}

/home/studio-lab-user/.conda/envs/dgl/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


该函数等价为`dgl.function.sum('m', 'h')`，其中第一个参数指定 mailbox 中的字段名，第二个参数表示目标节点特征的字段名。

`update_all`函数完整的调用过程如下，这个例子实现了将src（源节点）名为`ft`的feature和edge(边)名为`a`的feature相乘（u_mul_e）生成信息`m`，而后将节点收到的信息（也即`nodes.mailbox['m']`中的信息）相加，并存储为名为`ft`的feature。

In [3]:
import dgl.function as fn 
def update(graph):
    graph.update_all(fn.u_mul_e('ft', 'a', 'm'), fn.sum('m', 'ft'))

### multi_update_all

异构图上的消息传递可以分为两个部分：

（1）对每个关系计算和聚合消息，

（2）对每个结点聚合来自不同关系的消息。


在DGL中，对异构图进行消息传递的接口是 `multi_update_all`。 
`multi_update_all`接受一个字典。这个字典的每一个键值对里，键是一种关系， 值是这种关系对应`update_all`的参数。 
`multi_update_all`还接受一个字符串来表示跨类型整合函数，来指定整合不同关系聚合结果的方式。 
这个整合方式可以是 sum、 min、 max、 mean 和 stack 中的一个。以下是一个例子：

In [5]:
import dgl.function as fn
def multi_update_all(hgraph):
    for c_etype in hgraph.canonical_etypes:
        # 遍历异构图中所有的关系类型
        srctype, etype, dsttype = c_etype
        Wh = self.weight[etype](feat_dict[srctype])
        # 把它存在图中用来做消息传递
        hgraph.nodes[srctype].data['Wh_%s' % etype] = Wh
        # 指定每个关系的消息传递函数：(message_func, reduce_func).
        # 注意结果保存在同一个目标特征“h”，说明聚合是逐类进行的。
        funcs[etype] = (fn.copy_u('Wh_%s' % etype, 'm'), fn.mean('m', 'h'))
    # 将每个类型消息聚合的结果相加。
    hgraph.multi_update_all(funcs, 'sum')
    # 返回更新过的节点特征字典
    return {ntype : hgraph.nodes[ntype].data['h'] for ntype in hgraph.ntypes}